In [3]:
import numpy as np
from sklearn import metrics
import pandas as pd

In [34]:
directory = './data/'

In [35]:
r = 1
data = pd.DataFrame()
data['input'] = pd.read_csv(directory + 'english_tagalog.test', header=None)  # input sentence
data['score'] = pd.read_csv(directory + 'res_' + str(r) + '.txt', header=None).astype(np.float32)  # anomaly score
data['anomalous'] = 0  # anomalous or not
data

,input,score,anomalous
0,performing,0.000000,0
1,_the_part_,0.000000,0
2,i_did_besi,0.000000,0
3,des_cajoli,12.755096,0
4,ng_me_into,0.000000,0
...,...,...,...
827,a_nagasaka,0.000000,0
828,_panaog_pa,0.000000,0
829,kadto_sa_a,0.000000,0
830,kon_nga_an,0.000000,0


In [36]:
anomalous_data = data[data['anomalous'] == 1]
sensitivity = len(anomalous_data[anomalous_data['score'] > r]) / len(anomalous_data) * 100

non_anomalous_data = data[data['anomalous'] == 0]
specificity = len(non_anomalous_data[non_anomalous_data['score'] < r]) / len(non_anomalous_data) * 100
specificity = 1 - specificity

fpr, tpr, thresholds = metrics.roc_curve(specificity, sensitivity, pos_label=2)
metrics.auc(fpr, tpr)

ZeroDivisionError: division by zero